In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from ccfj import GetStationPairs
from geopy.distance import great_circle
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
#import shapefile
import geopandas as gp
import yaml
import math
#from pykml import parser


In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition

In [ ]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_17-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_image = dir_project + info_basic['dir_image']+'phase/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_partition = dir_project + info_basic['dir_partition']
dir_disp = dir_project + info_basic['dir_disp_model']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_all =  stainfo['latitude'].tolist() 
lon_all =  stainfo['longitude'].tolist() 

In [ ]:
stations = {}
lat = {}
lon = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations[key] = stations_this
    lat[key] = lat_stations_this.astype(float)
    lon[key] = lon_stations_this.astype(float)

In [ ]:
num_stations = []
for key_subwork in key_subworks:
    num_stations.append(len(stations[key_subwork]))

In [ ]:
#key_subworks = info_basic['key_subworks']
lat_centroid = []
lon_centroid = []
for key_subwork in key_subworks:
    lon_centroid.append(np.mean(lon[key_subwork]))
    lat_centroid.append(np.mean(lat[key_subwork]))

In [ ]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
#f_list = [3,3.5,4,4.5,5,10,15,20]
f_list = [7,10,15]
phase = {}
for f in f_list:
    phase[f] = []
for key in key_subworks:
    disp = np.loadtxt(dir_disp + 'disp_model_'+key+'.txt')
    disp = disp[disp[:,2] == 0]
    for f in f_list:
        phase[f].append(disp[disp[:,0]==f][0][1])

In [ ]:
size_range = [20,30]
num_range = [np.min(num_stations),np.max(num_stations)]
#将num_stations归一化到size_range
size = []
for num in num_stations:
    size.append(size_range[0]+(size_range[1]-size_range[0])*(num-num_range[0])/(num_range[1]-num_range[0]))

In [ ]:
"""
#ig,ax = plt.subplots(2,3,figsize=(15,8))
flag_this = 0
c_min = 0.4
c_max = 1.2
flag_num = 1
fig = plt.figure(figsize=(15,6))
for f in f_list:
    
    ax = plt.subplot(1,2,flag_num)

    ax.scatter(lon_all, lat_all,marker='.',color='k',s=1)
    im = ax.scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=size)
    #im = ax[0][flag_this].scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=200,vmin = c_min,vmax = c_max)
    plt.colorbar(im,ax=ax)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Phase Velocity at f = '+str(f)+' Hz (km/s)')
    ax.set_xticks([])
    ax.set_yticks([])
    for i in range(len(faults)):
        ax.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')


    flag_num += 1
plt.tight_layout()
print(dir_image+'phase_compare_'+str(f_list[0])+'_'+str(f_list[1])+'.png')
plt.savefig(dir_image+'phase_compare_'+str(f_list[0])+'_'+str(f_list[1])+'.png',dpi=300)
"""

In [ ]:
flag_this = 0
c_min = 0.4
c_max = 1.2
flag_num = 1
fig = plt.figure(figsize=(5*len(f_list),4))
for f in f_list:
    
    ax = plt.subplot(1,len(f_list),flag_num)

    ax.scatter(lon_all, lat_all,marker='.',color='k',s=1)
    im = ax.scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=size)
    #im = ax[0][flag_this].scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=200,vmin = c_min,vmax = c_max)
    plt.colorbar(im,ax=ax)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Phase Velocity at f = '+str(f)+' Hz (km/s)')
    ax.set_xticks([])
    ax.set_yticks([])
    for i in range(len(faults)):
        ax.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')


    flag_num += 1
plt.tight_layout()
print(dir_image+'phase_compare.png')
plt.savefig(dir_image+'phase_compare_2.png',dpi=300)

In [ ]:
"""
#ig,ax = plt.subplots(2,3,figsize=(15,8))
flag_this = 0
c_min = 0.4
c_max = 1.2
flag_num = 1
fig = plt.figure(figsize=(15,12))
for f in f_list:
    
    ax = plt.subplot(3,3,flag_num)

    ax.scatter(lon_all, lat_all,marker='.',color='k',s=1)
    im = ax.scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=size)
    #im = ax[0][flag_this].scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=200,vmin = c_min,vmax = c_max)
    plt.colorbar(im,ax=ax)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Phase Velocity at f = '+str(f)+' Hz (km/s)')
    ax.set_xticks([])
    ax.set_yticks([])
    for i in range(len(faults)):
        ax.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')


    flag_num += 1
plt.tight_layout()
print(dir_image+'phase_compare.png')
plt.savefig(dir_image+'phase_compare.png',dpi=300)
"""

In [ ]:
"""
fig,ax = plt.subplots(2,len(f_list),figsize=(5*len(f_list),8))
flag_this = 0
c_min = 0.4
c_max = 1.2
for f in f_list:
    ax[0][flag_this].scatter(lon_all, lat_all,marker='.',color='k')
    im = ax[0][flag_this].scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=30)
    #im = ax[0][flag_this].scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=200,vmin = c_min,vmax = c_max)
    plt.colorbar(im,ax=ax[0][flag_this])
    ax[0][flag_this].set_xlabel('Longitude')
    ax[0][flag_this].set_ylabel('Latitude')
    ax[0][flag_this].set_title('Phase Velocity at f = '+str(f)+' Hz (km/s)')
    ax[0][flag_this].set_xticks([])
    for i in range(len(faults)):
        ax[0][flag_this].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')

    ax[1][flag_this].scatter(lon_all, lat_all,marker='.',color='k')

    im = ax[1][flag_this].scatter(lon_centroid, lat_centroid,c=phase[f],cmap='jet',s=30,vmin = c_min,vmax = c_max)
    plt.colorbar(im,ax=ax[1][flag_this])
    ax[1][flag_this].set_xlabel('Longitude')
    ax[1][flag_this].set_ylabel('Latitude')
    ax[1][flag_this].set_title('Phase Velocity at f = '+str(f)+' Hz (km/s)')
    ax[1][flag_this].set_xticks([])
    for i in range(len(faults)):
        ax[1][flag_this].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')

    flag_this += 1
plt.tight_layout()
print(dir_image+'phase_compare.png')
plt.savefig(dir_image+'phase_compare.png',dpi=300)
"""